# Check LSLGA in DR8

#### John Moustakas

In [1]:
import os, pdb
import numpy as np
from glob import glob
import fitsio
from astropy.table import vstack, Table, hstack

In [2]:
import seaborn as sns
sns.set(context='talk', style='ticks', font_scale=1.6)
%matplotlib inline

In [4]:
dr8dir = '/global/project/projectdirs/cosmo/data/legacysurvey/dr8'
outdir = '/global/project/projectdirs/desi/users/ioannis/dr8-lslga'
lslgafile = '/global/project/projectdirs/cosmo/staging/largegalaxies/v2.0/LSLGA-v2.0.fits'

In [5]:
lslga = Table(fitsio.read(lslgafile))
#lslga

In [19]:
def build_lslga_dr8(northsouth, overwrite=False):
    cols = ['RA', 'DEC', 'TYPE', 'BRICKNAME', 'REF_CAT', 
            'FRACDEV', 'SHAPEEXP_R', 'SHAPEEXP_E1', 'SHAPEEXP_E2',
            'SHAPEDEV_R', 'SHAPEDEV_E1', 'SHAPEDEV_E2', 'REF_ID']

    outfile = os.path.join(outdir, 'dr8-lslga-{}.fits'.format(northsouth))
    if not os.path.isfile(outfile) or overwrite:
        out = []
        catfile = glob(os.path.join(dr8dir, northsouth, 'sweep', '8.0', 'sweep-*.fits'))
        #catfile = glob(os.path.join(dr8dir, northsouth, 'tractor', '???', 'tractor*.fits'))
        for ii, ff in enumerate(catfile):#[:3]):
            if ii % 20 == 0:
                print('{} / {}'.format(ii, len(catfile)))
            cc = fitsio.read(ff, columns=cols, upper=True)
            ckeep = np.where(cc['REF_CAT'] == b'L2')[0]
            out.append(Table(cc[ckeep]))
        out = vstack(out)
        
        # find and remove duplicates
        #uval, cnt = np.unique(out['REF_ID'], return_counts=True)
        #dup_refid = uval[cnt > 1]

        # now match against the LSLGA
        match = np.hstack([np.where(refid == lslga['LSLGA_ID'])[0] for refid in out['REF_ID']])
        
        lss = lslga[match]
        lss.rename_column('RA', 'RA_LSLGA')
        lss.rename_column('DEC', 'DEC_LSLGA')
        lss.rename_column('TYPE', 'MORPHTYPE')

        out = hstack((lss, out))
        
        print('Writing {}'.format(outfile))
        out.write(outfile, overwrite=True)
    else:
        print('Reading {}'.format(outfile))
        out = Table(fitsio.read(outfile))
    return out        

In [20]:
%time north = build_lslga_dr8('north', overwrite=False)

0 / 286
20 / 286
40 / 286
60 / 286
80 / 286
100 / 286
120 / 286
140 / 286
160 / 286
180 / 286
200 / 286
220 / 286
240 / 286
260 / 286
280 / 286
Writing /global/project/projectdirs/desi/users/ioannis/dr8-lslga/dr8-lslga-north.fits
CPU times: user 2min 44s, sys: 56.9 s, total: 3min 41s
Wall time: 4min 10s


In [22]:
%time south = build_lslga_dr8('south', overwrite=False)

0 / 437
20 / 437
40 / 437
60 / 437
80 / 437
100 / 437
120 / 437
140 / 437
160 / 437
180 / 437
200 / 437
220 / 437
240 / 437
260 / 437
280 / 437
300 / 437
320 / 437
340 / 437
360 / 437
380 / 437
400 / 437
420 / 437
Writing /global/project/projectdirs/desi/users/ioannis/dr8-lslga/dr8-lslga-south.fits
CPU times: user 9min 18s, sys: 3min 34s, total: 12min 53s
Wall time: 13min 52s


In [47]:
def merge_north_south(overwrite=False):
    mergedfile = os.path.join(outdir, 'dr8-lslga-northsouth.fits')
    if not os.path.isfile(mergedfile) or overwrite:
        out = []
        for northsouth in ('north', 'south'):
            catfile = os.path.join(outdir, 'dr8-lslga-{}.fits'.format(northsouth))
            cat = Table(fitsio.read(catfile))
            cat['REGION'] = northsouth
            out.append(cat)
        out = vstack(out)
        print('Writing {}'.format(mergedfile))
        out.write(mergedfile, overwrite=True)
    else:
        print('Reading {}'.format(mergedfile))
        out = Table(fitsio.read(mergedfile))
    return out

In [48]:
%time cat = merge_north_south(overwrite=True)

Writing /global/project/projectdirs/desi/users/ioannis/dr8-lslga/dr8-lslga-northsouth.fits
CPU times: user 16.5 s, sys: 426 ms, total: 16.9 s
Wall time: 17.6 s


In [49]:
cat

LSLGA_ID,GALAXY,PGC,RA_LSLGA,DEC_LSLGA,MORPHTYPE,BAR,RING,MULTIPLE,COMPACTNESS,T,PA,D25,BA,DIAM_REF,Z,SB_D25,MAG,MAG_REF,WISE_RA,WISE_DEC,CNTR,W1MPRO,W1SIGMPRO,W2MPRO,W2SIGMPRO,W3MPRO,W3SIGMPRO,W4MPRO,W4SIGMPRO,RCHI2,CC_FLAGS,EXT_FLG,PH_QUAL,XSCPROX,W1RSEMI,W1BA,W1PA,W1GMAG,W1GERR,W2GMAG,W2GERR,W3GMAG,W3GERR,W4GMAG,W4GERR,IN_ALLWISE,IN_DESI,NEAR_BRIGHTSTAR,BRICKNAME,TYPE,RA,DEC,FRACDEV,SHAPEDEV_R,SHAPEDEV_E1,SHAPEDEV_E2,SHAPEEXP_R,SHAPEEXP_E1,SHAPEEXP_E2,REF_CAT,REF_ID,REGION
int64,bytes29,int64,float64,float64,bytes4,bytes3,bytes3,bytes3,bytes3,float32,float32,float32,float32,bytes3,float32,float32,float32,bytes1,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float32,bytes4,int32,bytes4,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,bool,bool,bytes8,bytes4,float64,float64,float32,float32,float32,float32,float32,float32,float32,bytes2,int64,str5
14446,PGC1884023,1884023,141.4297125,29.9424196,E?,nan,nan,nan,nan,-5.0,94.47,0.36307806,0.36307806,iso,0.059890766,24.66048,18.232,B,141.4298823,29.9424617,1408130201351023996,13.668,0.025,13.434,0.032,10.085,0.062,8.421,0.314,1.954,0000,1,AAAB,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,True,True,False,1415p300,EXP,141.42986142163176,29.942504249477427,0.0,0.0,0.0,0.0,4.4602566,-0.57790524,-0.10098956,L2,14446,north
360448,PGC1885932,1885932,142.50464399999998,29.992207699999998,SBc,B,nan,nan,nan,4.9,36.43,0.4365158,0.8128305,iso,0.02685391,23.349482,16.521,B,142.5046815,29.9922294,1426130201351018332,13.98,0.028,13.857,0.041,10.689,0.104,8.44,nan,4.021,0000,5,AAAU,0.08,10.05,0.71,25.0,13.72,0.014,13.717,0.034,11.118,0.087,9.533,0.729,True,True,False,1424p300,DEV,142.50473664208823,29.992406310953097,1.0,52.381897,0.09154045,0.2133891,0.0,0.0,0.0,L2,360448,north
991749,PGC1867183,1867183,147.14676899999998,29.466061199999995,S?,nan,nan,nan,nan,3.0,88.48,0.56234133,0.22908677,iso,0.11317162,25.73748,18.359,B,147.1462573,29.4661434,1474128701351060944,14.084,0.033,13.755,0.042,10.772,0.119,8.672,nan,1.296,0000,0,AABU,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,True,True,False,1471p295,COMP,147.1468837545257,29.466073158523283,0.2651534,5.2139215,0.034630653,-0.0051687364,12.5653105,-0.8454735,0.09161035,L2,991749,north
936109,UGC04983,26553,140.72431650000001,29.3290403,SABc,B,nan,nan,D,5.0,5.3,0.6382635,0.8550667,iso,0.022450864,23.38848,15.735,B,140.724223,29.3290205,1405128701351058141,14.273,0.03,14.108,0.044,10.647,0.09,8.667,0.398,3.532,0000,5,AAAC,1.41,10.56,0.78,25.0,13.64,0.01,13.636,0.047,10.754,0.065,9.293,0.735,True,True,False,1406p292,COMP,140.7242657457912,29.32901976270882,0.015535885,0.64649796,-0.18028057,0.39984426,15.1308975,0.0042260797,0.033641696,L2,936109,north
1288301,PGC1868960,1868960,149.5192515,29.521296399999997,Sbc,nan,nan,nan,nan,3.8,78.87,0.33884412,0.53703177,iso,0.08934914,23.968481,17.69,B,149.519224,29.5213114,1491128701351055561,14.096,0.027,13.911,0.039,10.559,0.083,8.543,0.314,1.657,d000,0,AAAB,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,True,True,False,1494p295,EXP,149.5191878163415,29.52129808015057,0.0,0.0,0.0,0.0,4.19832,-0.39955872,0.17533746,L2,1288301,north
480891,IC2490,27121,143.26513500000001,29.9283201,SABb,B,nan,nan,nan,3.6,177.2,0.9660509,0.674528,iso,0.02446459,23.152481,14.599,B,143.2652455,29.92837,1426130201351023644,11.332,0.022,11.429,0.021,8.994,0.033,6.95,0.096,18.4,0000,5,AAAA,0.91,17.45,0.76,-5.0,10.977,0.006,10.973,0.008,8.411,0.014,6.713,0.073,True,True,False,1432p300,COMP,143.26517134982532,29.928353140597498,0.56626886,8.513987,0.21420135,-0.081731275,13.310233,0.15006962,0.03789386,L2,480891,north
1270041,SDSSJ095530.97+295527.8,4090945,148.8789615,29.9243707,nan,nan,nan,nan,nan,nan,145.74,0.33777565,0.43179256,r,0.25245062,25.000622,18.729,B,148.8788579,29.9244662,1495130201351013571,14.34,0.028,13.97,0.038,11.853,0.287,8.297,nan,1.739,0000,1,AABU,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,True,True,False,1487p

In [50]:
urefid, uindx = np.unique(cat['REF_ID'], return_index=True)
dup = np.delete(np.arange(len(cat)), uindx)
cat[dup]
cat[cat['REF_ID'] == 462171]

LSLGA_ID,GALAXY,PGC,RA_LSLGA,DEC_LSLGA,MORPHTYPE,BAR,RING,MULTIPLE,COMPACTNESS,T,PA,D25,BA,DIAM_REF,Z,SB_D25,MAG,MAG_REF,WISE_RA,WISE_DEC,CNTR,W1MPRO,W1SIGMPRO,W2MPRO,W2SIGMPRO,W3MPRO,W3SIGMPRO,W4MPRO,W4SIGMPRO,RCHI2,CC_FLAGS,EXT_FLG,PH_QUAL,XSCPROX,W1RSEMI,W1BA,W1PA,W1GMAG,W1GERR,W2GMAG,W2GERR,W3GMAG,W3GERR,W4GMAG,W4GERR,IN_ALLWISE,IN_DESI,NEAR_BRIGHTSTAR,BRICKNAME,TYPE,RA,DEC,FRACDEV,SHAPEDEV_R,SHAPEDEV_E1,SHAPEDEV_E2,SHAPEEXP_R,SHAPEEXP_E1,SHAPEEXP_E2,REF_CAT,REF_ID,REGION
int64,bytes29,int64,float64,float64,bytes4,bytes3,bytes3,bytes3,bytes3,float32,float32,float32,float32,bytes3,float32,float32,float32,bytes1,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float32,bytes4,int32,bytes4,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,bool,bool,bytes8,bytes4,float64,float64,float32,float32,float32,float32,float32,float32,float32,bytes2,int64,str5
462171,PGC1894795,1894795,222.76237500000002,30.2149495,E,nan,nan,M,nan,-5.0,15.19,0.6776415,0.699842,iso,0.12684074,24.591482,16.808,B,222.7623165,30.2149434,2226130201351032040,12.67,0.023,12.544,0.023,11.353,0.112,8.968,nan,4.878,0000,5,AABU,0.85,10.95,0.83,30.0,12.533,0.007,12.455,0.009,11.794,0.141,11.003,1.685,True,True,False,2226p302,REX,222.75788331496216,30.21070331309989,0.0,0.0,0.0,0.0,60.0,0.0,0.0,L2,462171,north
462171,PGC1894795,1894795,222.76237500000002,30.2149495,E,nan,nan,M,nan,-5.0,15.19,0.6776415,0.699842,iso,0.12684074,24.591482,16.808,B,222.7623165,30.2149434,2226130201351032040,12.67,0.023,12.544,0.023,11.353,0.112,8.968,nan,4.878,0000,5,AABU,0.85,10.95,0.83,30.0,12.533,0.007,12.455,0.009,11.794,0.141,11.003,1.685,True,True,False,2229p302,PSF,222.763145333835,30.214129860571248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,L2,462171,north
462171,PGC1894795,1894795,222.76237500000002,30.2149495,E,nan,nan,M,nan,-5.0,15.19,0.6776415,0.699842,iso,0.12684074,24.591482,16.808,B,222.7623165,30.2149434,2226130201351032040,12.67,0.023,12.544,0.023,11.353,0.112,8.968,nan,4.878,0000,5,AABU,0.85,10.95,0.83,30.0,12.533,0.007,12.455,0.009,11.794,0.141,11.003,1.685,True,True,False,2229p302,DEV,222.7622796253711,30.214941041938225,1.0,5.6969905,0.079771586,0.028879106,0.0,0.0,0.0,L2,462171,south


In [58]:
ww = [tt.strip() == 'REX' for tt in cat['TYPE']]
cat[ww]

LSLGA_ID,GALAXY,PGC,RA_LSLGA,DEC_LSLGA,MORPHTYPE,BAR,RING,MULTIPLE,COMPACTNESS,T,PA,D25,BA,DIAM_REF,Z,SB_D25,MAG,MAG_REF,WISE_RA,WISE_DEC,CNTR,W1MPRO,W1SIGMPRO,W2MPRO,W2SIGMPRO,W3MPRO,W3SIGMPRO,W4MPRO,W4SIGMPRO,RCHI2,CC_FLAGS,EXT_FLG,PH_QUAL,XSCPROX,W1RSEMI,W1BA,W1PA,W1GMAG,W1GERR,W2GMAG,W2GERR,W3GMAG,W3GERR,W4GMAG,W4GERR,IN_ALLWISE,IN_DESI,NEAR_BRIGHTSTAR,BRICKNAME,TYPE,RA,DEC,FRACDEV,SHAPEDEV_R,SHAPEDEV_E1,SHAPEDEV_E2,SHAPEEXP_R,SHAPEEXP_E1,SHAPEEXP_E2,REF_CAT,REF_ID,REGION
int64,bytes29,int64,float64,float64,bytes4,bytes3,bytes3,bytes3,bytes3,float32,float32,float32,float32,bytes3,float32,float32,float32,bytes1,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float32,bytes4,int32,bytes4,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,bool,bool,bool,bytes8,bytes4,float64,float64,float32,float32,float32,float32,float32,float32,float32,bytes2,int64,str5
315741,PGC1990555,1990555,229.19510400000001,32.302775800000006,nan,nan,nan,nan,nan,nan,24.0,0.3981072,0.52480745,iso,nan,24.078482,17.45,B,229.195031,32.3027161,2294131801351059978,14.295,0.055,14.389,0.065,13.069,nan,9.64,nan,1.002,0000,0,AAUU,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,True,True,False,2291p322,REX,229.1951220498187,32.30275951936196,0.0,0.0,0.0,0.0,0.21622105,0.0,0.0,L2,315741,north
462171,PGC1894795,1894795,222.76237500000002,30.2149495,E,nan,nan,M,nan,-5.0,15.19,0.6776415,0.699842,iso,0.12684074,24.591482,16.808,B,222.7623165,30.2149434,2226130201351032040,12.67,0.023,12.544,0.023,11.353,0.112,8.968,nan,4.878,0000,5,AABU,0.85,10.95,0.83,30.0,12.533,0.007,12.455,0.009,11.794,0.141,11.003,1.685,True,True,False,2226p302,REX,222.75788331496216,30.21070331309989,0.0,0.0,0.0,0.0,60.0,0.0,0.0,L2,462171,north
1371413,SDSSJ150625.84+305821.2,4012714,226.60772549999996,30.9726027,S?,nan,nan,nan,nan,10.0,30.49,0.37068075,0.8912509,iso,0.079420604,24.538483,18.065,B,226.6077065,30.9725723,2260130201351054762,15.55,0.036,15.372,0.071,11.992,0.174,9.307,0.485,1.053,0000,0,AABC,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,True,True,False,2267p310,REX,226.6076696598571,30.972539024099326,0.0,0.0,0.0,0.0,5.037455,0.0,0.0,L2,1371413,north
595523,PGC1922094,1922120,225.912018,30.882962899999995,Sc,nan,nan,nan,nan,4.7,nan,0.3935501,0.79250133,iso,0.06253826,23.53848,16.935,B,225.911424,30.8831464,2260130201351057318,14.078,0.041,14.015,0.05,10.557,0.068,8.348,nan,1.693,0000,1,AAAU,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,True,True,False,2258p310,REX,225.91197439733637,30.882926635706816,0.0,0.0,0.0,0.0,4.315515,0.0,0.0,L2,595523,north
840119,SDSSJ151418.44+345836.9,4012783,228.5768655,34.9769283,S?,nan,nan,nan,nan,10.0,147.27,0.35075188,0.8851156,iso,0.09033316,24.600481,18.247,B,228.5769649,34.9769404,2284134801351032314,15.645,0.036,15.527,0.077,12.23,0.222,9.699,nan,1.057,0000,0,AABU,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,True,True,False,2285p350,REX,228.57685627122453,34.97692087477085,0.0,0.0,0.0,0.0,5.431425,0.0,0.0,L2,840119,north
1294129,SDSSJ150553.63+334621.1,4005415,226.4734515,33.772516100000004,S?,nan,nan,nan,nan,10.0,nan,0.35809645,0.90573263,iso,0.0571035,24.379482,17.981,B,226.4736796,33.7726567,2267133301351042791,15.572,0.036,15.418,0.07,12.48,0.263,8.874,nan,1.147,0000,0,AABU,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,True,True,False,2263p337,REX,226.47354312175366,33.77256376179513,0.0,0.0,0.0,0.0,5.089376,0.0,0.0,L2,1294129,north
1145374,SDSSJ145933.78+325040.0,4541143,224.89074599999998,32.8444592,nan,nan,nan,nan,nan,nan,148.53,0.34197944,0.40457588,iso,0.008815098,24.61848,18.32,B,0.0,0.0,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,0,,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,False,True,False,2249p327,REX,224.89086803749296,32.844674146842756,0.0,0.0,0.0,0.0,1.9347694,0.0,0.0,L2,1145374,north
385434,SDSSJ151755.64+314119.0,4005695,229.4818335,31.688612199999998,Sc,nan,nan,M,nan,6.3,141.65,0.5345644,0.8035261,iso,0.034469176,24.34648,17.078,

#### Test out the code for adding this catalog as a separate layer

In [73]:
from astrometry.util.fits import fits_table
T = fits_table(os.path.join(outdir, 'dr8-lslga-northsouth.fits'))

In [96]:
def get_ba_pa(e1, e2):
    ee = np.hypot(e1, e2)
    ba = (1 - ee) / (1 + ee)
    #pa = -np.rad2deg(np.arctan2(e2, e1) / 2)
    pa = 180 - (-np.rad2deg(np.arctan2(e2, e1) / 2))
    return ba, pa

def lslga_model(T):
    ba, pa = [], []
    for Tone in T:
        ttype = Tone.type.strip()
        if ttype == 'DEV' or ttype == 'COMP':
            e1, e2 = Tone.shapedev_e1, Tone.shapedev_e2
            _ba, _pa = get_ba_pa(e1, e2)
        elif ttype == 'EXP' or ttype == 'REX':
            e1, e2 = Tone.shapeexp_e1, Tone.shapeexp_e2
            _ba, _pa = get_ba_pa(e1, e2)
        else: # PSF
            _ba, _pa = np.nan, np.nan 
        ba.append(_ba)
        pa.append(_pa)
    T.ba_model = np.hstack(ba)
    T.pa_model = np.hstack(pa)
    T.radius_model_arcsec = T.fracdev * T.shapedev_r + (1 - T.fracdev) * T.shapeexp_r
    T.radius_arcsec = T.d25 / 2. * 60.
    return T    

In [97]:
rr = lslga_model(T[:10])

In [98]:
rr.pa, rr.pa_model
rr.ba, rr.ba_model
rr.radius_arcsec, rr.radius_arcsec_model

(array([10.892342, 13.095474, 16.87024 , 19.147905, 10.165323, 28.981527,
        10.133269, 10.644402, 18.076786, 17.66531 ], dtype=float32),
 array([ 4.4602566, 52.381897 , 10.616065 , 14.9058695,  4.19832  ,
        10.594269 ,  4.775446 ,  3.208745 ,  8.014689 , 10.171474 ],
       dtype=float32))